Queue используется для обработки элементов в порядке их поступления, 
Deque позволяет работать с элементами как с начала, так и с конца

## 1)
Реализовать простейший динамический массив, поддерживающий добавление элемента в конец массива при помощи метода append(e).
Также должны поддерживаться следующие операции:

- извлечение элемента по индексу;
- устанавка нового значения по индексу;
- получение текущей длины массива.

1.1) Добавить реализацию удаления элемента из любого места массива и поддержку корректного вывода массива при помощи функции print.


In [3]:
class DynArray(object):
    def __init__(self):
        # на основе списка
        self.arr = []
    
    def append(self, element):
        self.arr.append(element)
    
    def my_get(self, index):
        if 0 <= index < len(self):
            return self.arr[index]
        return None

    def my_set(self, index, value):
        self.arr[index] = value
    
    def __len__(self):  # магический метод, который реализует len операцию
        # len(o) для классов list, dict, tuple, ...напрямую вызывает ф-ию на С
        # для остальных классов вызывает o.__len__()
        return len(self.arr)
    
    def delete(self, index):
        if 0 <= index < len(self):
            self.arr.pop(index)
    
    def print_arr(self, message=None):
        if message:
            print(message, end=': ')
        print(self.arr)


In [49]:
arr = DynArray()

arr.append(1)
arr.append(2)
arr.append(3)

arr.print_arr()

print('arr.get(1) ->', arr.my_get(1)) 
print('arr.get(10) ->', arr.my_get(10)) # не сущ-ет эл-та с таким индексом

arr.my_set(0,1000)
arr.print_arr('arr.set(0,1000)') 

print('len(arr) ->', len(arr))

arr.delete(0)
arr.print_arr('arr.delete(0)')
arr.delete(2)
arr.print_arr('arr.delete(2)')

[1, 2, 3]
arr.get(1) -> 2
arr.get(10) -> None
arr.set(0,1000): [1000, 2, 3]
len(arr) -> 3
arr.delete(0): [2, 3]
arr.delete(2): [2, 3]


## 2) 
Реализовать генератор, который возвращает значение поочередно извлекаемое из конца двух очередей (в качестве очереди используется deque из collections). Если очередь из которой извлекается элемент пуста - генератор заканчивает работу.

------------------------------------------------------------------------

Генератор создается по принципу обычной функции.

Отличие заключается в том, что вместо return используется инструкция yield.
Она уведомляет интерпретатор Python о том, что это генератор, и возвращает итератор.



In [4]:
#Синтаксис функции генератора:
def gen_func(args):
    ...
    while [cond]:
        ...
        yield [value]

In [2]:
from collections import deque

# dq = deque([iterable[, maxlen]])
#     iterable - итерируемая последовательность,
#     maxlen - int, максимальное кол-во хранимых записей.

# Deque.pop():
# удаляет и возвращает элемент с конца контейнера deque(). 
# Если элементы отсутствуют, возникает ошибка IndexError.

dq_1 = deque('abcd')
dq_2 = deque('123456')
print(dq_1, dq_2, sep='\n')
dq_3 = deque('123456')
dq_4 = deque('abcdef')
print(dq_3, dq_4, sep='\n')

def my_gen_func(dq1, dq2):
    flag = True
    go = True
    while go:
        if flag:  # удаление из dq1
            if not dq2: 
                go = False
            yield[dq1.pop()]
            flag = False
        else:  # удаление из dq2
            if not dq1: 
                go = False
            yield [dq2.pop()]
            flag = True


# другая версия
# yield - похож на return, но он НЕ прерывает выполнение генератора
def my_gen_func2(dq1, dq2):
    while dq1 and dq2:
        yield[dq1.pop()]
        yield [dq2.pop()]
        
            
my_generator = my_gen_func(dq_1, dq_2)
print('\n', my_generator)
print('dq1, dq2')
for el in my_generator:
    print(el, end='')
    
my_generator2 = my_gen_func2(dq_3, dq_4)
print('\n\n', my_generator2)
print('dq2, dq3')
for el in my_generator2:
    print(el, end='')

deque(['a', 'b', 'c', 'd'])
deque(['1', '2', '3', '4', '5', '6'])
deque(['1', '2', '3', '4', '5', '6'])
deque(['a', 'b', 'c', 'd', 'e', 'f'])

 <generator object my_gen_func at 0x00000190411DEFF0>
dq1, dq2
['d']['6']['c']['5']['b']['4']['a']['3']

 <generator object my_gen_func2 at 0x00000190411DF060>
dq2, dq3
['6']['f']['5']['e']['4']['d']['3']['c']['2']['b']['1']['a']

## 3)
Реализовать классы с медодом action():

а) Класс Pump - в методе action() извлекает очередное значение из генератора и помещает значение в очередь (очередь передается в конструктор).

b) Класс MultiAction - при вызове метода action() n раз вызвает метод action() класса, переданного в конструкторе. Число n также определяется в конструкторе.

c) Класс MultiPump - в методе action() извлекает очередное значение из генератора и помещает значение в одну из очередей (очереди передается в конструкторе); очереди , в которые помещаются очередные значения, меняются по порядку.


In [24]:
from queue import Queue

def my_gen():  # создаем простейший генератор
    i = 0
    while True:
        yield(i)
        i += 2

In [25]:
# a)

class Pump(object):
    def __init__(self, queue, generator):
        self.queue = queue
        self.generator = generator

    def action(self):
        value = next(self.generator)  # извлекает очередное значение из генератора 
        self.queue.put(value)

In [40]:
# b)

class MultiAction(object):
    def __init__(self, pumpclass_instance, n):
        self.pumpclass_instance = pumpclass_instance  # объект класса Pump
        self.n = n

    def action(self):
        # n раз вызвает метод action() класса Pump
        for _ in range(self.n):
            self.pumpclass_instance.action()  # сам вызов


In [47]:
# пример для class MultiAction

q = Queue()

pump1 = Pump(q, my_gen())
m_action = MultiAction(pump1, 10)  
m_action.action()
while not q.empty():
    print(q.get(), end=' ')

print('\n---------------')

pump2 = Pump(q, my_gen())
m_action = MultiAction(pump2, 2)  
m_action.action()
while not q.empty():
    print(q.get(), end=' ')

0 2 4 6 8 10 12 14 16 18 
---------------
0 2 

In [109]:
# c)

class MultiPump(object):
    def __init__(self, queues: list, generator):
        self.queues = queues  # набор очередей
        self.generator = generator
        self.current_queue = 0
    
    def action(self):
        value = next(self.generator)
        self.queues[self.current_queue].put(value)  # в выбранную очередь добавляем элемент
        self.current_queue = (self.current_queue + 1) % len(self.queues)

In [110]:
# пример для class MultiPump

queue1 = Queue()
queue2 = Queue()
        
m_pump = MultiPump([queue1, queue2], my_gen())

for _ in range(10):
    m_pump.action()

print('queue 1: ', end='')
while not queue1.empty():
    print(queue1.get(), end=' ')  # .get() - получаем, удаляем, возвращаем элемент из очереди  (начиная с начала)


queue 1: 0 4 8 12 16 

## 4) 
При помощи GenFromQ, Pump реализовать систему опработки сообщений.






In [121]:
# класс Pump для помещения сообщений в очередь - из предыдущего задания

# класс GenFromQ объединяет сообщения из нескольких очередей в одну
class GenFromQ(object):
    def __init__(self, queues):
        self.queues = queues

    def action(self):
        msg_lst = []
        for queue in self.queues:
            while not queue.empty():
                msg_lst.append(queue.get())
        return msg_lst


Сообщения создаются генератором сообщений возвращающим случайным образорм одно из сообщений. 

In [141]:
from queue import Queue
import random

# генератор случайных сообщений
def ran_msg_generator():
    msg = ['Hi!', 'meow meow', 'hello', 'bon jour', ':)']
    while True:
        yield random.choice(msg)


Сообщения из трех генераторов закачиваются в три очереди при помощи классов Pump, 

In [142]:
#три очереди и три объекта Pump для разных генераторов сообщений
queue1 = Queue()
queue2 = Queue()
queue3 = Queue()

pump1 = Pump(queue1, ran_msg_generator())
pump2 = Pump(queue2, ran_msg_generator())
pump3 = Pump(queue3, ran_msg_generator())

pump1.action()
pump1.action()
pump1.action()
pump2.action()
pump3.action()


далее при помощи GenFromQ и Pump объединяются в одну очередь 

In [143]:
gen_from_q = GenFromQ([queue1, queue2, queue3])

и выводятся на экран 

(можно реализовать при помощи класса с action и вызываемого при помощи MultiPump).

In [144]:

merged_messages = gen_from_q.action()

for msg in merged_messages:
    print(msg)

bon jour
hello
:)
meow meow
:)
